In [1]:
import numpy as np
import re

# Deep Learning for Natural Language and Code: Exercise 1

# Task 1: LIBSVM - BOW

## Load data

1. Download the dataset from [here](https://ai.stanford.edu/%7Eamaas/data/sentiment)
1. Copy the dataset next to this Jupyter (.ipynb file)
1. Install:
    * Sklearn (This library is only allowed to use for reading the BOW in LIBSVM format)

A **LIBSVM file** is a plain text file format used to store **sparse datasets** for machine learning tasks, especially **classification** and **regression**. It's called "LIBSVM" because it was originally used by the **LIBSVM** library, a very popular library for Support Vector Machines.

The format looks like this:

```
<label> <index1>:<value1> <index2>:<value2> <index3>:<value3> ...
```

- `<label>` = the **target value** (for example, `1` for positive, `-1` for negative).
- `<index>:<value>` = the **non-zero features**.
  - `<index>` is the feature number (starting at 1),
  - `<value>` is the value of that feature (usually the count or some preprocessed weight).

If a feature is **zero**, it is simply **omitted** from the line (to save space — this is why it's called *sparse* format).

---

### A real small example:

Suppose we have two movie reviews turned into a Bag-of-Words (BoW):
- Feature 1 = "awesome"
- Feature 2 = "terrible"
- Feature 3 = "boring"
- Feature 4 = "amazing"

And two reviews:
- Review 1 (positive): "awesome amazing"
- Review 2 (negative): "terrible boring boring"

The LIBSVM file would look like:

```
1 1:1 4:1
-1 2:1 3:2
```

**Explanation:**
- The first line:
  - `1` → label is positive
  - `1:1` → "awesome" appeared once
  - `4:1` → "amazing" appeared once
- The second line:
  - `-1` → label is negative
  - `2:1` → "terrible" appeared once
  - `3:2` → "boring" appeared twice

---

### Why use LIBSVM format?

- It's super lightweight for huge datasets where most feature values are 0.
- It's easy to parse and generate manually.
- Many machine learning tools accept this format directly (e.g., SVM, Random Forest, logistic regression).

**What's inside `./aclImdb/train/labeledBow.feat` ?**

- There are 25 000 lines
- One line per sample
- first element of each line is the label (frol 1 to 10)
- others are the number of apparences of each feature (e.g `0:9 1:1` means feature 0 appeared 9 times and the feature 1 appeared only once)


### Warning on dense matrix sizes

My custom code initially constructs a dense matrix to represent the libsvm file. 
- [ ] Compute the size needed in memory to store the dense matrix 




In [2]:
from typing import cast

type Feature = int
type Occurrence = int
type Label = int


def parse_libsvm_line(line: str) -> tuple[list[tuple[Feature, Occurrence]], Label]:
    label, features = line.split(" ", 1)
    features = features.split(" ")
    features = cast(
        list[tuple[int, int]],
        [tuple(map(int, feature.split(":"))) for feature in features],
    )
    return features, int(label)


assert parse_libsvm_line("1 0:9 1:1 14:87") == ([(0, 9), (1, 1), (14, 87)], 1)


def parse_libsvm_content(content: str) -> tuple[np.ndarray, np.ndarray]:
    data = [parse_libsvm_line(line) for line in content.split("\n") if line]

    vocabulary_size = (
        max((feature for features, _ in data for feature, _ in features), default=0) + 1
    )
    X = np.zeros((len(data), vocabulary_size))
    y = np.zeros(len(data))
    for i, (line, label) in enumerate(data):
        for feature, occurrence in line:
            X[i, feature] = occurrence
        y[i] = label
    return X, y


X, y = parse_libsvm_content("1 0:9 1:1 3:87\n-1 2:1 3:2")
assert np.all(X == [[9, 1, 0, 87], [0, 0, 1, 2]])
assert np.all(y == [1, -1])


def load_libsvm_file(path: str) -> tuple[np.ndarray, np.ndarray]:
    with open(path, "r") as f:
        return parse_libsvm_content(f.read())

In [3]:
X, y = load_libsvm_file("./aclImdb/train/labeledBow.feat")

In [ ]:
X

In [ ]:
y

In [6]:
from sklearn.datasets import load_svmlight_file


# X_sklearn, y_sklearn = load_svmlight_file('./aclImdb/train/labeledBow.feat')
# Commented out because it's slow

# assert X_sklearn.shape == X.shape
# assert np.allclose(X_sklearn.todense(), X)
# assert np.allclose(y_sklearn, y)
# # ->  Good, my implementation is consistent with the sklearn implementation

### Vocabulary

In [ ]:
from pathlib import Path


VOCAB_FILE = Path("./aclImdb/imdb.vocab")
assert VOCAB_FILE.exists()


def read_vocab(file: Path) -> list[str]:
    return file.read_text().splitlines()


vocab = read_vocab(VOCAB_FILE)

vocab[:10]

In [8]:
# Read the the bag of words and the Y for the training data
X, y = load_libsvm_file("./aclImdb/train/labeledBow.feat")
# Read the the bag of words and the Y for the test data
X_test, y_test = load_libsvm_file("./aclImdb/test/labeledBow.feat")

### Expected Rating

In [ ]:
ratings = [float(line) for line in open("./aclImdb/imdbEr.txt")]
ratings[:10]

In [ ]:
for token, rating in list(zip(vocab, ratings))[:10]:
    print(f"{token}: {rating}")

In [ ]:
# find the most negative and most positive words
most_negative_words = sorted(zip(ratings, vocab), key=lambda x: x[0])[:10]
most_positive_words = sorted(zip(ratings, vocab), key=lambda x: x[0])[-10:]

print("Most negative words:")
for rating, word in most_negative_words:
    print(f"{word}: {rating}")

print("\nMost positive words:")
for rating, word in most_positive_words:
    print(f"{word}: {rating}")

# Task 2: Bag of Words (BOW)
## Load Raw text and scores 

1. Be sure to have downloaded the dataset from the link provided in the exercise and have read the README file
1. Be sure to have copied the dataset next to this Jupyter (.ipynb file)
1. Be sure to have installed:
    * Numpy
    * NLTK (only for the stemming process)
    * Sklearn (only for building a Random Forest)
1. In this part of the exercise it is not allowed to use Sklearn
1. Build the Bag Of Words (BOW) with the raw data, for this you need to:
    * Tokenize on spaces and punctuation
    * Lower case
    * Remove punctuation
    * Remove terms appearing more often than X percent, this X percent should be variable. Which means that you should be able to change the percentage as a parameter.
    * Use NLTK porter stemmer
1. Build a classifier with the BOW previously built. Take into account:
    * The RF should be a binary classification positive (i.e., score >=7) and negative (i.e., score <= 4)
    * Test the classifier with the test data

##  Load data

In [12]:
TRAIN_POS_DIR = Path("./aclImdb/train/pos")
TRAIN_NEG_DIR = Path("./aclImdb/train/neg")
TEST_POS_DIR = Path("./aclImdb/test/pos")
TEST_NEG_DIR = Path("./aclImdb/test/neg")

assert (
    TRAIN_POS_DIR.exists()
    and TRAIN_NEG_DIR.exists()
    and TEST_POS_DIR.exists()
    and TEST_NEG_DIR.exists()
)
assert (
    TRAIN_POS_DIR.is_dir()
    and TRAIN_NEG_DIR.is_dir()
    and TEST_POS_DIR.is_dir()
    and TEST_NEG_DIR.is_dir()
)

In [14]:
all_texts: list[str] = []

for dir in TRAIN_POS_DIR, TRAIN_NEG_DIR:
    for file in dir.glob("*.txt"):
        id, label = map(int, file.name.strip(".txt").split("_"))

        assert 0 <= id <= 12499
        assert 1 <= label <= 10

        text = file.read_text()
        all_texts.append(text)

assert len(all_texts) == 25000

In [15]:
from typing import Sequence

# I manually set this list of characters to remove
# after I detected them in the following cells.
# But I could use https://pypi.org/project/Unidecode/ instead
CHARACTERS_TO_REMOVE = [
    "\x96",
    "\x91",
    "\x97",
    "\xad",
    "\x84",
    "\x08",
    "\x80",
    "\x8e",
    "\x9e",
    "\x95",
    "\x9a",
    "\x10",
    "\x8d",
    "\uf0b7",
]


def clean_text_from_weird_chars(
    text: str,
    *,
    weird_chars: Sequence[str] = CHARACTERS_TO_REMOVE,
    replace_with: str = " ",
) -> str:
    for char in weird_chars:
        text = text.replace(char, replace_with)
    return text


all_texts = [clean_text_from_weird_chars(text) for text in all_texts]

### Analyze lenghts of reviews 

In [ ]:
import matplotlib.pyplot as plt


def plot_text_length_distribution(texts: list[str]) -> None:
    lengths = np.array([len(text) for text in texts])
    plt.figure(figsize=(10, 6))
    plt.hist(lengths, bins=50, color="skyblue", edgecolor="black")
    plt.title("Distribution of Text Lengths (in characters)")
    plt.xlabel("Text Length (characters)")
    plt.ylabel("Number of Reviews")
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.show()


plot_text_length_distribution(all_texts)

In [ ]:
def get_shortest_and_longest_reviews(
    texts: list[str], n: int = 5
) -> tuple[list[str], list[str]]:
    """
    Returns the n shortest and n longest reviews from the given list of texts.

    Args:
        texts: List of text reviews
        n: Number of shortest and longest reviews to return

    Returns:
        tuple containing lists of shortest and longest reviews
    """
    # Create a list of (text, length) tuples
    text_lengths = [(text, len(text)) for text in texts]

    # Sort by length
    text_lengths.sort(key=lambda x: x[1])

    # Get n shortest and n longest
    shortest = [text for text, _ in text_lengths[:n]]
    longest = [text for text, _ in text_lengths[-n:]]

    return shortest, longest


# Get 5 shortest and 5 longest reviews
shortest_reviews, longest_reviews = get_shortest_and_longest_reviews(all_texts, 5)

# Display shortest reviews
print("5 SHORTEST REVIEWS:")
for i, review in enumerate(shortest_reviews, 1):
    print(f"\nShortest Review #{i} ({len(review)} characters):")
    print("-" * 50)
    print(review[:200] + "..." if len(review) > 200 else review)
    print("-" * 50)

# Display longest reviews
print("\n\n5 LONGEST REVIEWS:")
for i, review in enumerate(longest_reviews, 1):
    print(f"\nLongest Review #{i} ({len(review)} characters):")
    print("-" * 50)
    # Only show the beginning and end of very long reviews
    if len(review) > 400:
        print(review[:200] + "\n...\n" + review[-200:])
    else:
        print(review)
    print("-" * 50)


In [ ]:
# import re
# def find_non_alphanumeric(texts: list[str]) -> dict[str, int]:
#     non_alphanumeric = {}
#     for text in texts:
#         for char in text:
#             if not re.match(r'[A-Za-z0-9\s]', char):
#                 non_alphanumeric[char] = non_alphanumeric.get(char, 0) + 1
#     return non_alphanumeric

# Find and display non-alphanumeric characters in the reviews
def find_non_alphanumeric(texts: list[str]) -> dict[str, int]:
    """
    Finds all non-alphanumeric characters in the texts and counts their occurrences.

    Args:
        texts: List of text reviews

    Returns:
        Dictionary mapping non-alphanumeric characters to their occurrence counts
    """
    non_alphanumeric = {}

    for text in texts:
        for char in text:
            if char == "½":
                non_alphanumeric[char] = non_alphanumeric.get(char, 0) + 1
            elif not char.isalnum() and not char.isspace():
                non_alphanumeric[char] = non_alphanumeric.get(char, 0) + 1

    return non_alphanumeric


assert find_non_alphanumeric(["½"]) == {"½": 1}


# Get non-alphanumeric characters and their counts
non_alphanumeric_chars = find_non_alphanumeric(all_texts)

# Sort by frequency (most common first)
sorted_chars = sorted(non_alphanumeric_chars.items(), key=lambda x: x[1], reverse=True)

# Display the results
print("NON-ALPHANUMERIC CHARACTERS (sorted by frequency):")
print("-" * 50)
for char, count in sorted_chars:
    print(f"'{char}': {count} occurrences")
print("-" * 50)

# Show some examples of reviews with special characters
print("\nEXAMPLES OF REVIEWS WITH SPECIAL CHARACTERS:")
for char, _ in sorted_chars:  # Take top 5 most common special chars
    for text in all_texts[:1000]:  # Search in first 1000 reviews for efficiency
        if char in text:
            print(f"\nReview containing '{char}':")
            print("-" * 50)
            # Find the position of the character and show context around it
            pos = text.find(char)
            start = max(0, pos - 50)
            end = min(len(text), pos + 50)
            context = text[start:end]
            # Highlight the character
            print(f"...{context.replace(char, f'[{char}]')}...")
            print("-" * 50)
            break  # Only show one example per character


**Results:**


Here’s an analysis and categorization of the non-alphanumeric characters you listed, with explanations and examples for each group:

---

**Punctuation (Standard English)**
These are the most common and serve grammatical or stylistic purposes in English text.

- **Sentence-ending:**  
  - `.` (period), `!` (exclamation), `?` (question mark)
- **Pausing/Separating:**  
  - `,` (comma), `;` (semicolon), `:` (colon), `-` (hyphen), `–` (en dash), `—` (em dash)
- **Quotation/Dialogue:**  
  - `'` (apostrophe), `"` (double quote), `‘’` (curly single quotes), `“”` (curly double quotes)
- **Parenthetical/Grouping:**  
  - `(`, `)`, `[`, `]`, `{`, `}`
- **Other:**  
  - `/` (slash), `\\` (backslash), `|` (pipe), `*` (asterisk), `&` (ampersand), `#` (hash), `@` (at), `^` (caret), `_` (underscore), `+` (plus), `=` (equals), `%` (percent), `$` (dollar), `~` (tilde), `` ` `` (backtick)



**HTML/XML Markup**
Characters commonly found in HTML tags or entities, often due to the dataset containing raw web data.

- `<`, `>`, `/`  
  - Used for opening/closing tags: `<br />`, `<p>`, etc.
  - **Also used for the heart symbol "<3" : this should not be cleaned away !**

**Special/Extended ASCII and Unicode**
These include typographic symbols, currency, and accented characters, often from non-English text or encoding artifacts.

- **Accented/Foreign Letters:**  
  - `´`, `¨`, `¡`, `£`, `¤`, `®`, `°`, ``, `₤`, `¿`
- **Typographic/Formatting:**  
  - `…` (ellipsis), `·` (middle dot), `§` (section), `¦` (broken bar), `–` (en dash), `—` (em dash), `“”` (curly quotes), `‘’` (curly apostrophes), `«»` (guillemets)

**Mathematical/Technical Symbols**
Occasionally appear in reviews, especially in ratings or technical discussions.

- `*`, `+`, `=`, `%`, `#`, `^`, `|`, `/`, `\\`

**Currency and Miscellaneous**
- `$`, `£`, `₤` (currencies)
- `@` (email, social media handles)
- `~` (approximation, tilde)
- `°` (degree symbol)

**Rare/Unusual Characters**
- `¿`, `¡` (Spanish punctuation)
- `»`, `«` (French/Spanish quotes)
- `¤`, `®`, `¦`, `§`, `·`, `¨`, `…` (various typographic or currency symbols)

**Whitespace and Control Characters**
Not explicitly listed, but often present in text data (e.g., `\n`, `\t`, non-breaking space).


**NLP Implications**

- **Punctuation** is often removed or normalized in preprocessing, but apostrophes and hyphens may be important for meaning (e.g., “don’t”, “re-enter”).
- **HTML/XML** should be stripped or converted to plain text.
- **Encoding Artifacts** should be cleaned or normalized to avoid noise.
- **Special/Unicode** characters may need normalization, especially for multilingual data.
- **Currency and Technical Symbols** may be relevant for certain tasks (e.g., financial sentiment).

In [ ]:
### Get html tags

import re
from collections import defaultdict

# Dictionary to track occurrences of HTML-like tags
html_tag_occurrences = defaultdict(int)
html_tag_examples = defaultdict(list)

# Find all potential HTML tags in the texts
for text in all_texts:
    # Find all occurrences of patterns that look like HTML tags
    potential_tags = re.findall(r"<[^>]+>", text)

    for tag in potential_tags:
        html_tag_occurrences[tag] += 1
        # Store up to 3 examples of context for each tag
        if len(html_tag_examples[tag]) < 3:
            # Get some context around the tag
            start_idx = max(0, text.find(tag) - 30)
            end_idx = min(len(text), text.find(tag) + len(tag) + 30)
            context = text[start_idx:end_idx]
            html_tag_examples[tag].append(context)

# Display the most common HTML-like tags and their examples
print("Most common HTML-like tags:")
for tag, count in sorted(
    html_tag_occurrences.items(), key=lambda x: x[1], reverse=True
)[:500]:
    print(f"{tag}: {count} occurrences")
    print("Examples:")
    for example in html_tag_examples[tag]:
        print(f"  - ...{example}...")
    print()

# Also check for individual < and > characters that might not be part of tags
less_than_count = sum("<" in text for text in all_texts)
greater_than_count = sum(">" in text for text in all_texts)

print(f"Total texts with '<' character: {less_than_count}")
print(f"Total texts with '>' character: {greater_than_count}")


In [73]:
def clean_html_tags(text: str) -> str:
    """
    Remove HTML tags from text while preserving the content between tags.
    Special case: preserves "<3" (heart symbol).

    Args:
        text: The input text containing potential HTML tags

    Returns:
        Text with HTML tags removed but content preserved
    """
    import re

    # First, temporarily replace "<3" with a special marker
    text = text.replace("<3", "HEART_SYMBOL_PLACEHOLDER")

    # Remove HTML tags
    cleaned_text = re.sub(r"<[^>]*>", "", text)

    # Restore the heart symbols
    cleaned_text = cleaned_text.replace("HEART_SYMBOL_PLACEHOLDER", "<3")

    return cleaned_text


# Test basic HTML tag removal
assert clean_html_tags("<p>Hello world</p>") == "Hello world"

# Test nested tags
assert clean_html_tags("<div><p>Nested content</p></div>") == "Nested content"

# Test with attributes
assert clean_html_tags('<a href="https://example.com">Link text</a>') == "Link text"

# Test with multiple tags and text
assert clean_html_tags("<h1>Title</h1><p>Paragraph</p>") == "TitleParagraph"

# Test with the special case of "<3" (heart symbol)
assert (
    clean_html_tags(
        "I LUVED IT SO MUCH <3 <br /><br />its about a women...<br /><br /> her<br /><br />"
    )
    == "I LUVED IT SO MUCH <3 its about a women... her"
)

# Test with mixed content
assert (
    clean_html_tags("Text with <b>bold</b> and <i>italic</i>")
    == "Text with bold and italic"
)

# Test with "<em>"
assert clean_html_tags("<em>This is emphasized</em>") == "This is emphasized"

# Test with </SPOILER>
assert clean_html_tags("</SPOILER>This is a spoiler</SPOILER>") == "This is a spoiler"

# Test with empty tags
assert clean_html_tags("<br><hr>Text") == "Text"

# Test with no tags
assert clean_html_tags("Plain text without tags") == "Plain text without tags"

assert any(
    "<3" in text for text in all_texts
)  # check that the heart symbol is still in the text

In [74]:
all_texts = [clean_html_tags(text) for text in all_texts]

assert any(
    "<3" in text for text in all_texts
)  # check that the heart symbol is still in the text

In [75]:
CHAR_MAP = {
    "%": " ",  # -> replace it with a space (will be tokenized later)
    "´": "'",  # -> Unify apostrophes. Apostrophes are important in words like "don't"
    ")": " ",
    "}": " ",
    "-": None,  # --> will be removed or kept later, depending on the context
    "/": " ",
    "$": " ",
    "_": " ",  # in the dataset, it's used to make text in italics, so we remove it since that's only formatting
    "£": " ",
    "\\": " ",
    "'": "'",
    "“": " ",
    "~": " ",
    "¦": " ",
    "»": " ",
    "^": " ",
    "¨": " ",
    "(": " ",
    "”": " ",
    "|": " ",
    "’": "'",
    ".": " ",
    "[": " ",
    "°": " ",
    "¡": " ",
    "·": " ",
    "!": " ",
    "+": " ",
    "¤": " ",
    "¿": " ",
    ";": " ",
    "{": " ",
    '"': " ",
    "?": " ",
    "<": " ",
    ">": " ",
    "–": " ",
    "®": " ",
    "*": " ",
    "=": " ",
    "#": " ",
    "]": " ",
    "…": " ",
    ":": " ",
    ",": " ",
    "&": " ",
    "₤": " ",
    "‘": "'",
    "§": " ",
    "`": " ",
    "@": " ",
    "«": " ",
    "½": " ",
    "¢": " ",
    "©": " ",
    "″": " ",
    "，": " ",
    "、": " ",
    "★": " ",
    "▼": " ",
}


def get_rid_of_non_alphanumeric_characters(text: str, char_map: dict[str, str]) -> str:
    """
    Remove non-alphanumeric characters from text based on the provided character map.

    Args:
        text: The input text to clean
        char_map: A dictionary mapping characters to their replacements. If the replacement is None, the character will be kept as is.

    Returns:
        Text with non-alphanumeric characters replaced as specified in the character map.
    """

    for char, replacement in char_map.items():
        if replacement is None:
            continue  # keep the character as is
        else:
            text = text.replace(char, replacement)

    return text


assert get_rid_of_non_alphanumeric_characters("Hello world", CHAR_MAP) == "Hello world"
assert (
    get_rid_of_non_alphanumeric_characters("Hello @ world", CHAR_MAP) == "Hello   world"
)
assert (
    get_rid_of_non_alphanumeric_characters(
        "only £300 000 and 7 weeks to write.", CHAR_MAP
    )
    == "only  300 000 and 7 weeks to write "
)


def keep_or_remove_dashes(text: str) -> str:
    """
    Processes dashes in a text string based on their context.

    Keeps dashes that appear to be part of a word, specifically those
    immediately surrounded by alphanumeric characters. Replaces all
    other dashes (e.g., standalone dashes, dashes next to spaces,
    consecutive dashes not surrounded by alphanumeric characters)
    with a single space.

    Args:
        text: The input text string.

    Returns:
        The processed text string where dashes not part of words are
        replaced by spaces.

    Examples:
        >>> keep_or_remove_dash("a-composed-word")
        'a-composed-word'
        >>> keep_or_remove_dash("an hyphen in - the - middle - of a word")
        'an hyphen in   the   middle   of a word'
        >>> keep_or_remove_dash(" - this is a bullet list but this-is-a-composed-word")
        '   this is a bullet list but this-is-a-composed-word'
        >>> keep_or_remove_dash("multiple---consecutive---dashes")
        'multiple   consecutive   dashes'
    """
    result = []
    n = len(text)
    for i, char in enumerate(text):
        if char == "-":
            # Check if the dash is part of a word (surrounded by alphanumeric chars)
            is_part_of_word = (
                i > 0 and text[i - 1].isalnum() and i < n - 1 and text[i + 1].isalnum()
            )

            if is_part_of_word:
                result.append("-")
            else:
                # Replace dash with space if it's not part of a word
                result.append(" ")
        else:
            result.append(char)
    return "".join(result)


assert keep_or_remove_dashes("a-composed-word") == "a-composed-word"
assert (
    keep_or_remove_dashes("an hyphen in - the - middle - of a word")
    == "an hyphen in   the   middle   of a word"
)
assert (
    keep_or_remove_dashes(" - this is a bullet list but this-is-a-composed-word")
    == "   this is a bullet list but this-is-a-composed-word"
)

In [76]:
all_texts = [
    get_rid_of_non_alphanumeric_characters(text, CHAR_MAP) for text in all_texts
]

all_texts = [keep_or_remove_dashes(text) for text in all_texts]

In [ ]:
print("Remaining non-alphanumeric characters:")
find_non_alphanumeric(all_texts)

In [ ]:
char = "-"
for text in [text for text in all_texts if char in text][:50]:
    # Find all positions where the character appears
    positions = [i for i, c in enumerate(text) if c == char]

    if positions:
        # For each position, show context (5 characters before and after)
        for pos in positions:
            start = max(0, pos - 10)
            end = min(len(text), pos + 10)  # +6 to include the character at pos+5
            context = text[start:end]
            print(f"...{context}...")

### Lowercase the text

In [16]:
all_texts = [text.lower() for text in all_texts]

### Tokenize text

Since we already replaced all unwanted and punctuation characters with spaces, we only need to split by spaces ! 

In [17]:
def tokenize_and_clean_tokens(text: str) -> list[str]:
    """
    Splits text into tokens by whitespace and cleans individual tokens
    (e.g., removes leading/trailing apostrophes).
    """
    tokens = text.split()
    cleaned_tokens = []
    for token in tokens:
        # Remove leading/trailing apostrophes that might remain
        cleaned_token = token.strip("'")
        # Remove trailing 's that might remain
        cleaned_token = cleaned_token.rstrip("'s")

        if cleaned_token:  # Avoid empty strings
            cleaned_tokens.append(cleaned_token)

    assert all(
        not re.search(r"\s", t) for t in cleaned_tokens
    )  # check that there is no whitespace in the tokens

    return cleaned_tokens


assert tokenize_and_clean_tokens("'Hello'") == ["Hello"]
assert tokenize_and_clean_tokens("'Hello' world") == ["Hello", "world"]
assert tokenize_and_clean_tokens("'Hello' world") == ["Hello", "world"]


In [ ]:
all_tokens = [t for text in all_texts for t in tokenize_and_clean_tokens(text)]
all_unique_tokens = set(all_tokens)

print(f"{len(all_tokens)=}")
print(f"Unique tokens : {len(all_unique_tokens)}")

In [ ]:
sorted(list(all_unique_tokens))[:100]

In [ ]:
[t for t in list(all_unique_tokens) if "'" in t or "’" in t or '"' in t][:100]

In [84]:
with open("all_unique_tokens.txt", "w") as f:
    for token in sorted(all_unique_tokens):
        f.write(f"{token}\n")


### Tokens frequency

In [ ]:
from collections import Counter

tokens_counter: Counter[str] = Counter(all_tokens)

# Get the 15 most common tokens
most_common_tokens = tokens_counter.most_common(15)
print("15 most common tokens (corpus frequency):")
for token, _ in most_common_tokens:
    print(token)

# Get the 15 least common tokens
least_common_tokens = tokens_counter.most_common()[:-16:-1]
print("\n15 least common tokens (corpus frequency):")
for token, _ in least_common_tokens:
    print(token)


In [ ]:
from collections import Counter

# Count document frequency using Counter
tokens_counter: Counter[str] = Counter()

for text in all_texts:
    doc_tokens = set(tokenize_and_clean_tokens(text))
    tokens_counter.update(doc_tokens)

# Show the 15 most frequent tokens by document frequency
most_common_df_tokens = tokens_counter.most_common(15)
print("15 most common tokens (document frequency):")
for token, freq in most_common_df_tokens:
    print(f"{token}: {freq}")

In [ ]:
from wordcloud import WordCloud


def plot_wordcloud(tokens_counts: Counter[str]) -> None:
    wc = WordCloud(width=800, height=400, background_color="white")
    wc.generate_from_frequencies(tokens_counts)
    plt.figure(figsize=(12, 6))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title("Token Frequency Word Cloud")
    plt.show()


plot_wordcloud(tokens_counter)

In [ ]:
# Define different threshold percentages for removing most frequent tokens
threshold_percentages = [
    0.01,
    0.02,
    0.05,
    0.1,
    0.2,
    0.5,
    1,
    5,
    10,
]  # Remove top X% most frequent tokens

# Create a figure with subplots for each threshold
plt.figure(figsize=(15, 12))

for i, THRESHOLD_PERCENTAGE in enumerate(threshold_percentages, 1):
    # Calculate how many tokens to remove
    total_unique_tokens = len(tokens_counter)
    n_tokens_to_remove = int(total_unique_tokens * THRESHOLD_PERCENTAGE / 100)

    print(f"\n--- Threshold: {THRESHOLD_PERCENTAGE}% ---")
    print(f"Total unique tokens: {total_unique_tokens}")
    print(
        f"Removing top {THRESHOLD_PERCENTAGE}% ({n_tokens_to_remove}) most frequent tokens"
    )

    # Get the tokens to remove (the most frequent ones)
    tokens_to_remove = [
        token for token, _ in tokens_counter.most_common(n_tokens_to_remove)
    ]

    # Create a new counter without the removed tokens
    filtered_tokens_counter = Counter(
        {
            token: count
            for token, count in tokens_counter.items()
            if token not in tokens_to_remove
        }
    )

    print(f"Tokens remaining after filtering: {len(filtered_tokens_counter)}")

    # Compare the original and filtered token counts
    print(f"Original total token count: {sum(tokens_counter.values())}")
    print(f"Filtered total token count: {sum(filtered_tokens_counter.values())}")
    print(
        f"Percentage of tokens removed: {(sum(tokens_counter.values()) - sum(filtered_tokens_counter.values())) / sum(tokens_counter.values()) * 100:.2f}%"
    )

    # Display some of the removed tokens
    print("Sample of removed tokens (top 5):")
    for token in tokens_to_remove[:5]:
        print(f"'{token}' (count: {tokens_counter[token]})")

    # Create a subplot for this threshold
    # Dynamically determine subplot grid size based on number of thresholds
    import math

    n_thresholds = len(threshold_percentages)
    n_cols = math.ceil(math.sqrt(n_thresholds))
    n_rows = math.ceil(n_thresholds / n_cols)
    plt.subplot(n_rows, n_cols, i)

    # Generate and display wordcloud
    wc = WordCloud(width=800, height=400, background_color="white")
    wc.generate_from_frequencies(filtered_tokens_counter)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"Word Cloud - {THRESHOLD_PERCENTAGE}% threshold")

plt.tight_layout()
plt.show()


In [ ]:
# Display the most frequent tokens after filtering with 5% threshold (as an example)
THRESHOLD_PERCENTAGE = 1
n_tokens_to_remove = int(total_unique_tokens * THRESHOLD_PERCENTAGE / 100)
tokens_to_remove = {
    token for token, _ in tokens_counter.most_common(n_tokens_to_remove)
}
filtered_tokens_counter = Counter(
    {
        token: count
        for token, count in tokens_counter.items()
        if token not in tokens_to_remove
    }
)

print(
    f"\nMost frequent tokens after filtering with {THRESHOLD_PERCENTAGE}% threshold (top 10):"
)
for token, count in filtered_tokens_counter.most_common(10):
    print(f"'{token}' (count: {count})")

filtered_unique_tokens = {
    token for token in tokens_counter if token not in tokens_to_remove
}

In [ ]:
with open("filtered_tokens.txt", "w") as f:
    for token in sorted(filtered_unique_tokens):
        f.write(f"{token}\n")

list(sorted(filtered_unique_tokens))[:10]


### Porter Stemmer

In [ ]:
from typing import Iterable
from nltk.stem import PorterStemmer


def stem_words(words: Iterable[str], stemmer: PorterStemmer | None = None) -> list[str]:
    if not stemmer:
        stemmer = PorterStemmer()

    return [stemmer.stem(word) for word in words]


stem_words(["running", "runs", "ran", "easily", "fairly"])

### Tokenize tokens

In [ ]:
# Number of tokens before stemming :
print(
    f"Number of unique tokens before stemming, after removing {THRESHOLD_PERCENTAGE}% most common : {len(filtered_unique_tokens)}"
)

In [ ]:
stemmed_unique_tokens: set[str] = set(stem_words(filtered_unique_tokens))

print(f"Number of unique tokens after stemming : {len(stemmed_unique_tokens)}")


for token in list(stemmed_unique_tokens)[:10]:
    print(f"{token}")


In [94]:
with open("tokens.txt", "w") as f:
    for token in sorted(stemmed_unique_tokens):
        f.write(f"{token}\n")


## Build the bag of words (BOW)
For building the matrix for the representation of bag of words use the previously built vocabulary and tokens for each review.

In [ ]:
final_bow_vocabulary_list = sorted(list(stemmed_unique_tokens))
final_bow_vocabulary_map = {
    token: i for i, token in enumerate(final_bow_vocabulary_list)
}
print(f"Final BoW vocabulary size: {len(final_bow_vocabulary_list)}")

# Task 3: Comparing BOWs

1. Use the previous steps to build a bag of words with the training data in which the tokens that appear more than 1% are discarded. 
1. Compare your BOW with LIBSVM BOW. 

# Task 4: Train a Random Forest and test it


In [96]:
from sklearn.ensemble import RandomForestClassifier

# Task 5: Markov chain
Tip: For memory optimization use sparse structures not a matrix mostrly filled with zeros

## Pre-process data
Read the data and using the previous built functions for the BOW representation create a list of words per each review

## Chain words
Identify all the possible pairs of words (w0, w1) in all the reviews

## Initialize the Markov's Chain

## Generate data

Here you could also try to generate words for the unlabeled part of the dataset. Try to meassure the quality of the model

## Theoretical Questions

### 1. Categorization of Tasks

Define whether Tasks 2, 4, and 5 fall under NLG, NLU, or NLP.

### 2. Text Processing Pipeline Discussion

Discuss the advantages and drawbacks of steps like stemming and punctuation removal.

### 3. Review Linguistic Concepts

Define:

- Polysemy
- Zeugma
- Homonyms
- Homographs
- Homophones

Then answer:

- Are these concepts important in NLP? Why?
- How do they impact a BoW representation?

### 4. Markov Model Performance

Think about the performance of the Markov model, is it good? Why or why not?